# Deep Learning Fundus Image Analysis for Early Detection of Diabetic Retinopathy
## Training Xception Model

This notebook trains a deep learning model using Xception architecture for classifying diabetic retinopathy severity levels.

## Step 1: Create Training and Testing Path

Assign variables for train and test paths. The Xception model requires input images of size 299x299.

In [ ]:
# Training and Testing Path Configuration
TRAIN_PATH = '../data'
TEST_PATH = '../data'

# Xception Model Configuration
IMG_SIZE = 299  # Xception input size is 299x299
BATCH_SIZE = 64
EPOCHS = 30
NUM_CLASSES = 5

print(f"Training Path: {TRAIN_PATH}")
print(f"Testing Path: {TEST_PATH}")
print(f"Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Epochs: {EPOCHS}")
print(f"Number of Classes: {NUM_CLASSES}")

## Step 2: Import Required Libraries

Import all necessary libraries for building and training the deep learning model.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
import os

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## Step 3: Configure ImageDataGenerator Class

ImageDataGenerator is used for data augmentation. We apply various transformations to increase dataset diversity:
- Image shifts (width_shift_range, height_shift_range)
- Image flips (horizontal_flip, vertical_flip)
- Image rotations (rotation_range)
- Image brightness (brightness_range)
- Image zoom (zoom_range)

In [ ]:
# Training Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,                    # Normalize pixel values to [0,1]
    rotation_range=20,                 # Image rotations (0-20 degrees)
    width_shift_range=0.2,             # Image shifts horizontally (20%)
    height_shift_range=0.2,            # Image shifts vertically (20%)
    horizontal_flip=True,              # Image flips horizontally
    vertical_flip=True,                # Image flips vertically
    zoom_range=0.2,                    # Image zoom (20%)
    shear_range=0.2,                   # Shear transformation
    brightness_range=[0.8, 1.2],       # Image brightness adjustment
    fill_mode='nearest',               # Fill mode for new pixels
    validation_split=0.2               # 20% for validation
)

# Testing Data - Only Rescaling (No Augmentation)
test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

print("Data Augmentation Configured Successfully!")
print("Training augmentation includes: rotation, shift, flip, zoom, shear, brightness")
print("Testing data: only rescaling applied")

## Step 4: Apply ImageDataGenerator to Train and Test Sets

Use flow_from_directory() to load images from subdirectories.

**Arguments:**
- directory: Path where data is located
- target_size: Size to resize images (299x299 for Xception)
- batch_size: Number of images per batch (64)
- class_mode: 'categorical' for multi-class classification

In [ ]:
# Training Set
train_generator = train_datagen.flow_from_directory(
    directory=TRAIN_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

# Validation/Test Set
test_generator = test_datagen.flow_from_directory(
    directory=TEST_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

print(f"\nClass Indices: {train_generator.class_indices}")
print(f"Training Samples: {train_generator.samples}")
print(f"Validation Samples: {test_generator.samples}")
print(f"Steps per Epoch: {train_generator.samples // BATCH_SIZE}")
print(f"Validation Steps: {test_generator.samples // BATCH_SIZE}")

## Step 5: Pre-trained CNN Model as Feature Extractor

Load Xception model pre-trained on ImageNet as a feature extractor.
- include_top=False: Exclude fully connected layers
- weights='imagenet': Use pre-trained weights
- Freeze all convolution blocks to prevent weight updates

In [ ]:
# Load Xception Pre-trained Model
base_model = Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# Freeze all layers in base model (Feature Extraction)
base_model.trainable = False

print(f"Base Model: Xception")
print(f"Input Shape: {IMG_SIZE}x{IMG_SIZE}x3")
print(f"Pre-trained Weights: ImageNet")
print(f"Include Top: False (using as feature extractor)")
print(f"Trainable: False (all layers frozen)")
print(f"Total Base Model Layers: {len(base_model.layers)}")

## Step 6: Adding Dense Layers

Add custom dense layers on top of the Xception base model.
The output layer has neurons equal to the number of classes (5) with softmax activation.

In [ ]:
# Build Custom Classification Layers
x = base_model.output
x = GlobalAveragePooling2D(name='global_avg_pooling')(x)
x = Dense(512, activation='relu', name='dense_512')(x)
x = Dropout(0.5, name='dropout_1')(x)
x = Dense(256, activation='relu', name='dense_256')(x)
x = Dropout(0.3, name='dropout_2')(x)
predictions = Dense(NUM_CLASSES, activation='softmax', name='output')(x)

# Create Final Model
model = Model(inputs=base_model.input, outputs=predictions)

print("Custom Layers Added:")
print("  1. GlobalAveragePooling2D")
print("  2. Dense(512, activation='relu')")
print("  3. Dropout(0.5)")
print("  4. Dense(256, activation='relu')")
print("  5. Dropout(0.3)")
print(f"  6. Dense({NUM_CLASSES}, activation='softmax') - Output Layer")

## Step 7: Model Summary

Display the complete model architecture with layer details.

In [ ]:
model.summary()

## Step 8: Configure the Learning Process

Compile the model with:
- Optimizer: Adam (adaptive learning rate)
- Loss Function: categorical_crossentropy (for multi-class classification)
- Metrics: accuracy

In [ ]:
# Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Model Compiled Successfully!")
print("  - Optimizer: Adam (learning_rate=0.0001)")
print("  - Loss Function: categorical_crossentropy")
print("  - Metrics: accuracy")

## Step 9: Setup Callbacks

Configure callbacks for training:
- ModelCheckpoint: Save best model based on validation loss
- EarlyStopping: Stop training if no improvement
- ReduceLROnPlateau: Reduce learning rate when metric plateaus

In [ ]:
# ModelCheckpoint: Save the best model
checkpoint = ModelCheckpoint(
    'Updated-Xception-diabetic-retinopathy.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)

# EarlyStopping: Stop if no improvement
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    restore_best_weights=True
)

# ReduceLROnPlateau: Reduce learning rate
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-7
)

callbacks = [checkpoint, early_stop, reduce_lr]

print("Callbacks Configured:")
print("  1. ModelCheckpoint - Save best model")
print("  2. EarlyStopping - Stop if no improvement for 5 epochs")
print("  3. ReduceLROnPlateau - Reduce LR by 0.5 if plateau for 3 epochs")

## Step 10: Train the Model

Train the model using fit() method.

**Arguments:**
- steps_per_epoch: Total training samples / batch_size
- epochs: Number of training iterations
- validation_data: Test generator for validation
- validation_steps: Total validation samples / batch_size

In [ ]:
# Calculate steps
steps_per_epoch = train_generator.samples // BATCH_SIZE
validation_steps = test_generator.samples // BATCH_SIZE

print(f"Training Configuration:")
print(f"  - Steps per Epoch: {steps_per_epoch}")
print(f"  - Validation Steps: {validation_steps}")
print(f"  - Total Epochs: {EPOCHS}")
print(f"\nStarting Training...\n")

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    validation_data=test_generator,
    validation_steps=validation_steps,
    callbacks=callbacks,
    verbose=1
)

## Step 11: Save the Model

Save the trained model in HDF5 (.h5) format.
H5 file contains multidimensional arrays of scientific data including model architecture and weights.

In [ ]:
# Save Model
model_path = 'Updated-Xception-diabetic-retinopathy.h5'
model.save(model_path)

print(f"Model saved successfully to: {model_path}")
print(f"Model file format: HDF5 (.h5)")
print(f"File contains: Architecture + Weights + Optimizer state")

## Step 12: Evaluate the Model

Evaluate the trained model on the test set and display performance metrics.

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=validation_steps)

print(f"\nFinal Results:")
print(f"  - Test Loss: {test_loss:.4f}")
print(f"  - Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# Training history summary
print(f"\nTraining History:")
print(f"  - Final Training Loss: {history.history['loss'][-1]:.4f}")
print(f"  - Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"  - Final Validation Loss: {history.history['val_loss'][-1]:.4f}")
print(f"  - Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"  - Best Validation Loss: {min(history.history['val_loss']):.4f}")
print(f"  - Best Validation Accuracy: {max(history.history['val_accuracy']):.4f}")

## Visualize Training History

In [ ]:
# Plot training history
plt.figure(figsize=(14, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print("Training Complete!")